In [1]:
# import necessary libraries
import pandas as pd
import pandas as pd
import numpy as np
import ast
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
playerMatchDetailsPerformanceDF = pd.read_csv('../aggregateMatchAndTimelineLarge.csv')
summonerRanksDF = pd.read_csv('../summonerRanksLarge.csv')
summonerRanksDF = summonerRanksDF[['summonerId', 'queueType', 'tier', 'rank', 'leaguePoints']] # just selecting necessary sets of data

print(playerMatchDetailsPerformanceDF.shape)
print(summonerRanksDF.shape)

for aCol in playerMatchDetailsPerformanceDF.columns:
    print(aCol)

print("-------------------------------")
for rCol in summonerRanksDF.columns:
    print(rCol)

(204990, 29)
(85758, 5)
gameId
gameDuration
puuid
summonerId
teamId
win
teamPosition
champExperience
kills
assists
turretKills
epicMonsterKills
visionScore
visionWardsBoughtInGame
magicDamageDealtToChampions
deaths
totalMinionsKilled
timeCCingOthers
totalDamageTaken
totalHealsOnTeammates
totalTimeSpentDead
goldEarned
objectivesStolen
objectivesStolenAssists
participantId
participantsAssisted
towerKillsAssisted
monsterKillsAssisted
participantsAssistedWithPressure
-------------------------------
summonerId
queueType
tier
rank
leaguePoints


In [3]:
# join summonerRanks with aggregateMatchDF
playerMatchDetailsPerformanceWithRanksDF = playerMatchDetailsPerformanceDF.merge(summonerRanksDF, on='summonerId', how='inner')
print(playerMatchDetailsPerformanceWithRanksDF.shape)

# count of total players in each game; both teams; can be used for filtering matches or check sanity tests
matchPlayerCounts = playerMatchDetailsPerformanceWithRanksDF.groupby(['gameId'])['summonerId'].count().reset_index()
matchPlayerCounts.rename(columns={'summonerId': 'matchPlayerCounts'}, inplace=True)

# adds the matchPlayerCounts to the dataframe in use
playerMatchDetailsPerformanceWithRanksDF = playerMatchDetailsPerformanceWithRanksDF.merge(matchPlayerCounts, on='gameId', how='inner')
playerMatchDetailsPerformanceWithRanksDF.head()

# apply the filter of 10 players in a team to prevent cases of a single player being AFK
valid_player_match_details_df = playerMatchDetailsPerformanceWithRanksDF[playerMatchDetailsPerformanceWithRanksDF['matchPlayerCounts'] == 10]
print("Number of matches after player count filter: ", valid_player_match_details_df.shape[0] / 10) # divide by 10 because there are 10 rows for each match; corresponding to players



(165324, 33)
Number of matches after player count filter:  3489.0


In [4]:
# of the 7220 player records (i.e. 722 matches for 10 players in each team), the RECORDS for RANKED games should total to this number of 7220
valid_player_match_details_df['queueType'].value_counts()

## here 5838 + 1385 added to 7220. don't be confused that RANKED SOLO and RANKED FLEX always has to be multiple of 10. 
# it is possible that RANKED SOLO is being matched with opponent of RANKED FLEX 5


queueType
RANKED_SOLO_5x5    28247
RANKED_FLEX_SR      6643
Name: count, dtype: int64

In [5]:
valid_player_match_details_df['summonerId'].value_counts()

summonerId
u0ktvlKPXElSgQKqk0Xedq7s95ckFQx8geWKo63yVdiewEU               23
TB0oTrM2N4YGOod55J-g79YtEZX-Nd436QJq6Ii-uICxrlU               21
_GbZWlwO9f9F9FcJKC4l5QOXYbbfBA2R7MIjvUax2B6f5-mmlJQ4rdJ4uw    21
O1YTv743Fco5dCLzfHDXygivUT_dEUBRkULAAhYXY3nIUqQ               21
lZyx8pvmKZF-zntJjRvkJ47DARff0c1iJLR6ksguG-d0TiQ               20
                                                              ..
k0hhbrMvNk4VMhp3_e_8FhJToklYvDGNYnYF6rbw7IZPxps                1
pzsOc7AxuhQ_UCP6rXuFAJ5De_E3zjZiB8q60ztFZ6gnkqdkEGljkRs4dA     1
L9d3TxYwYfMabdQOg6tWMv4iuRT2gmKT2FCf9WUU98UZK2D1               1
D34HOh7mXvPD42WLTZeAu6SD4DwHhcZf7CyivsP-tR9fOD9QZcOgD5MeCw     1
jOCexYmhP5FPWfLsH4BfP420EOQoQeOOnf0Df7TbDy-4SuA                1
Name: count, Length: 26612, dtype: int64

In [6]:
# convert player ranks to numerical ranks
# join the tier and rank to yield a combined value
valid_player_match_details_df['tierRank'] = valid_player_match_details_df['tier'] + '-' + valid_player_match_details_df['rank']

rankedDict = {
    'NA':0,
    'IRON-IV':1,
    'IRON-III':2,
    'IRON-II':3,
    'IRON-I':4,
    'BRONZE-IV':5,
    'BRONZE-III':6,
    'BRONZE-II':7,
    'BRONZE-I':8,
    'SILVER-IV':9,
    'SILVER-III':10,
    'SILVER-II':11,
    'SILVER-I':12,
    'GOLD-IV':13,
    'GOLD-III':14,
    'GOLD-II':15,
    'GOLD-I':16,
    'PLATINUM-IV':17,
    'PLATINUM-III':18,
    'PLATINUM-II':19,
    'PLATINUM-I':20,
    'EMERALD-IV':21,
    'EMERALD-III':22,
    'EMERALD-II':23,
    'EMERALD-I':24,
    'DIAMOND-IV':25,
    'DIAMOND-III':26,
    'DIAMOND-II':27,
    'DIAMOND-I':28,
    'MASTER-I':29,
    'GRANDMASTER-I':30,
    'CHALLENGER-I':31
}

# for calculating a numerical rank value for a player of a given match
def calculateNumericalRank(row):
    return rankedDict[row['tierRank']]

valid_player_match_details_df['playerNumericalRank'] = valid_player_match_details_df.apply(calculateNumericalRank, axis=1)

# averaging the rank of all 10 players in a match
matchAverageRankDF = valid_player_match_details_df.groupby(['gameId'])['playerNumericalRank'].mean().reset_index()
matchAverageRankDF.rename(columns={'playerNumericalRank': 'matchAverageRank'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(matchAverageRankDF, on='gameId', how='inner')

# averaging the rank of a given team in a match; 5 players in the team
teamAverageRankDF = valid_player_match_details_df.groupby(['gameId', 'teamId'])['playerNumericalRank'].mean().reset_index()
teamAverageRankDF.rename(columns={'playerNumericalRank': 'teamAverageRank'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(teamAverageRankDF, on=['gameId', 'teamId'], how='inner')

# summing up total goldEarned by a team in a match
totalTeamGold = valid_player_match_details_df.groupby(['gameId', 'teamId'])['goldEarned'].sum().reset_index()
totalTeamGold.rename(columns={'goldEarned': 'totalTeamGold'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamGold, on=['gameId', 'teamId'], how='inner')

# another column for total gold per minute for the team
valid_player_match_details_df['totalTeamGPM'] = np.ceil(valid_player_match_details_df['totalTeamGold'] / valid_player_match_details_df['gameDuration'])

# summing up the total kills by a team in a match
totalTeamKills = valid_player_match_details_df.groupby(['gameId', 'teamId'])['kills'].sum().reset_index()
totalTeamKills.rename(columns={'kills': 'totalTeamKills'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamKills, on=['gameId', 'teamId'], how='inner')

# another column for total kills per minute for the team
valid_player_match_details_df['totalTeamKPM'] = valid_player_match_details_df['totalTeamKills'] / valid_player_match_details_df['gameDuration']

# summing up the total deaths of a team in a match
totalTeamDeaths = valid_player_match_details_df.groupby(['gameId', 'teamId'])['deaths'].sum().reset_index()
totalTeamDeaths.rename(columns={'deaths': 'totalTeamDeaths'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamDeaths, on=['gameId', 'teamId'], how='inner')

# summing up the total assists by a team in a match
totalTeamAssists = valid_player_match_details_df.groupby(['gameId', 'teamId'])['assists'].sum().reset_index()
totalTeamAssists.rename(columns={'assists': 'totalTeamAssists'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamAssists, on=['gameId', 'teamId'], how='inner')

valid_player_match_details_df['totalTeamAPM'] = valid_player_match_details_df['totalTeamAssists'] / valid_player_match_details_df['gameDuration']

# summing up the total epicMonsterKills by a team in a match
totalTeamEpicMonsterKills = valid_player_match_details_df.groupby(['gameId', 'teamId'])['epicMonsterKills'].sum().reset_index()
totalTeamEpicMonsterKills.rename(columns={'epicMonsterKills': 'totalTeamEpicMonsterKills'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamEpicMonsterKills, on=['gameId', 'teamId'], how='inner')

# summing up the total towers taken by a team in a match
totalTeamTowerKills = valid_player_match_details_df.groupby(['gameId', 'teamId'])['turretKills'].sum().reset_index()
totalTeamTowerKills.rename(columns={'turretKills': 'totalTeamTurretKills'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamTowerKills, on=['gameId', 'teamId'], how='inner')

# summing up vision score for a team in a match
totalTeamVisionScore = valid_player_match_details_df.groupby(['gameId', 'teamId'])['visionScore'].sum().reset_index()
totalTeamVisionScore.rename(columns={'visionScore': 'totalTeamVisionScore'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamVisionScore, on=['gameId', 'teamId'], how='inner')

# another metric for vision score of the team per minute
valid_player_match_details_df['totalTeamVPM'] = valid_player_match_details_df['totalTeamVisionScore'] / valid_player_match_details_df['gameDuration']

# team average vision score in a match
averageTeamVisionScore = valid_player_match_details_df.groupby(['gameId', 'teamId'])['visionScore'].mean().reset_index()
averageTeamVisionScore.rename(columns={'visionScore': 'averageTeamVisionScore'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(averageTeamVisionScore, on=['gameId', 'teamId'], how='inner')

# team total champ experience in a match
totalTeamChampExperience = valid_player_match_details_df.groupby(['gameId', 'teamId'])['champExperience'].sum().reset_index()
totalTeamChampExperience.rename(columns={'champExperience': 'totalTeamChampExperience'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamChampExperience, on=['gameId', 'teamId'], how='inner')

# another column for total champ experience per minute per team
valid_player_match_details_df['totalTeamExpPerMin'] = valid_player_match_details_df['totalTeamChampExperience'] / valid_player_match_details_df['gameDuration']

# average champ experience per team
averageTeamChampExperience = valid_player_match_details_df.groupby(['gameId', 'teamId'])['champExperience'].mean().reset_index()
averageTeamChampExperience.rename(columns={'champExperience': 'averageTeamChampExperience'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(averageTeamChampExperience, on=['gameId', 'teamId'], how='inner')

# total minions killed by a team in a match
totalTeamMinionsKilled = valid_player_match_details_df.groupby(['gameId', 'teamId'])['totalMinionsKilled'].sum().reset_index()
totalTeamMinionsKilled.rename(columns={'totalMinionsKilled': 'totalTeamMinionsKilled'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(totalTeamMinionsKilled, on=['gameId', 'teamId'], how='inner')

valid_player_match_details_df['totalTeamMinionsPerMin'] = valid_player_match_details_df['totalTeamMinionsKilled'] / valid_player_match_details_df['gameDuration']

# average minions killed per team
averageTeamMinionsKilled = valid_player_match_details_df.groupby(['gameId', 'teamId'])['totalMinionsKilled'].mean().reset_index()
averageTeamMinionsKilled.rename(columns={'totalMinionsKilled': 'averageTeamMinionsKilled'}, inplace=True)
valid_player_match_details_df = valid_player_match_details_df.merge(averageTeamMinionsKilled, on=['gameId', 'teamId'], how='inner')

print(valid_player_match_details_df.shape)

valid_player_match_details_df[['gameId', 'teamId', 'win', 'participantId', 'teamPosition', 'kills', 'deaths', 'epicMonsterKills','assists', 'turretKills', 'playerNumericalRank', 'teamAverageRank', 'matchAverageRank', 'totalTeamKills', 'totalTeamDeaths', 'totalTeamAssists', 'totalTeamEpicMonsterKills', 'totalTeamTurretKills']].head(20)

/tmp/ipykernel_71766/1140947639.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_player_match_details_df['tierRank'] = valid_player_match_details_df['tier'] + '-' + valid_player_match_details_df['rank']
/tmp/ipykernel_71766/1140947639.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_player_match_details_df['playerNumericalRank'] = valid_player_match_details_df.apply(calculateNumericalRank, axis=1)


(34890, 56)


,gameId,teamId,win,participantId,teamPosition,kills,deaths,epicMonsterKills,assists,turretKills,playerNumericalRank,teamAverageRank,matchAverageRank,totalTeamKills,totalTeamDeaths,totalTeamAssists,totalTeamEpicMonsterKills,totalTeamTurretKills
0,EUW1_7056642171,100,0,1,TOP,7,8,0,2,2,13,13.0,13.8,39,41,38,1,7
1,EUW1_7056642171,100,0,2,JUNGLE,8,8,1,11,1,13,13.0,13.8,39,41,38,1,7
2,EUW1_7056642171,100,0,3,MIDDLE,10,9,0,4,1,14,13.0,13.8,39,41,38,1,7
3,EUW1_7056642171,100,0,4,BOTTOM,10,9,0,9,3,12,13.0,13.8,39,41,38,1,7
4,EUW1_7056642171,100,0,5,UTILITY,4,7,0,12,0,13,13.0,13.8,39,41,38,1,7
5,EUW1_7056642171,200,1,6,TOP,3,10,0,9,4,14,14.6,13.8,41,39,69,4,8
6,EUW1_7056642171,200,1,7,JUNGLE,13,7,3,13,0,22,14.6,13.8,41,39,69,4,8
7,EUW1_7056642171,200,1,8,MIDDLE,7,10,1,14,0,13,14.6,13.8,41,39,69,4,8
8,EUW1_7056642171,200,1,9,BOTTOM,16,8,0,10,4,13,14.6,13.8,41,39,69,4,8
9,EUW1_7056642171,200,1,10,UTILITY,2,4,0,23,0,11,14.6,13.8,41,39,69,4,8


In [7]:
for col in valid_player_match_details_df.columns:
    print(col)

gameId
gameDuration
puuid
summonerId
teamId
win
teamPosition
champExperience
kills
assists
turretKills
epicMonsterKills
visionScore
visionWardsBoughtInGame
magicDamageDealtToChampions
deaths
totalMinionsKilled
timeCCingOthers
totalDamageTaken
totalHealsOnTeammates
totalTimeSpentDead
goldEarned
objectivesStolen
objectivesStolenAssists
participantId
participantsAssisted
towerKillsAssisted
monsterKillsAssisted
participantsAssistedWithPressure
queueType
tier
rank
leaguePoints
matchPlayerCounts
tierRank
playerNumericalRank
matchAverageRank
teamAverageRank
totalTeamGold
totalTeamGPM
totalTeamKills
totalTeamKPM
totalTeamDeaths
totalTeamAssists
totalTeamAPM
totalTeamEpicMonsterKills
totalTeamTurretKills
totalTeamVisionScore
totalTeamVPM
averageTeamVisionScore
totalTeamChampExperience
totalTeamExpPerMin
averageTeamChampExperience
totalTeamMinionsKilled
totalTeamMinionsPerMin
averageTeamMinionsKilled


In [8]:
# create a sub-set of the dataframe from the valid matches that could be used to map assists
assistsMapDF = valid_player_match_details_df[[
    'gameId', 'teamId', 'win', 'teamPosition', 'teamAverageRank', 'matchAverageRank', # overall match info 
    'participantId', 'kills', 'deaths', 'assists', 'participantsAssisted', 'towerKillsAssisted', 'monsterKillsAssisted', 'participantsAssistedWithPressure', # participant performance
    'totalTeamKills', 'totalTeamAssists', 'totalTeamTurretKills', 'totalTeamEpicMonsterKills', # team performance
    'totalTeamGPM', 'totalTeamKPM', 'totalTeamExpPerMin', 'totalTeamVPM', 'totalTeamAPM', 'totalTeamMinionsPerMin' # per min team performance
]]

# convert the string representation of assists list into actual lists using ast
assistsMapDF['participantsAssisted'] = assistsMapDF['participantsAssisted'].apply(ast.literal_eval)
assistsMapDF['towerKillsAssisted'] = assistsMapDF['towerKillsAssisted'].apply(ast.literal_eval)
assistsMapDF['monsterKillsAssisted'] = assistsMapDF['monsterKillsAssisted'].apply(ast.literal_eval)
assistsMapDF['participantsAssistedWithPressure'] = assistsMapDF['participantsAssistedWithPressure'].apply(ast.literal_eval)

# concatenate the 4 lists into a single list
assistsMapDF['totalAssisted'] = assistsMapDF.apply(lambda row: [item for sublist in (row['participantsAssisted'], row['towerKillsAssisted'], row['monsterKillsAssisted'], row['participantsAssistedWithPressure']) for item in sublist], axis=1)

# new column to store the count of pressure assists per team
totalTeamPressureAssists = assistsMapDF.groupby(['teamId', 'gameId'])['participantsAssistedWithPressure'].apply(lambda x: x.apply(len).sum()).reset_index()
totalTeamPressureAssists.rename(columns={'participantsAssistedWithPressure': 'totalTeamPressureAssists'}, inplace=True)
assistsMapDF = assistsMapDF.merge(totalTeamPressureAssists, on=['gameId', 'teamId'], how='inner')

# new column that sums up team assists of all kinds; used for normalization
assistsMapDF['totalTeamAllAssists'] = assistsMapDF['totalTeamAssists'] + assistsMapDF['totalTeamEpicMonsterKills'] + assistsMapDF['totalTeamTurretKills'] + assistsMapDF['totalTeamPressureAssists']
assistsMapDF['totalTeamAllKills'] = assistsMapDF['totalTeamKills'] + assistsMapDF['totalTeamTurretKills'] + assistsMapDF['totalTeamEpicMonsterKills']
assistsMapDF['intensity'] = assistsMapDF['totalTeamAllAssists'] / assistsMapDF['totalTeamAllKills']

assistsMapDF.head(20)

/tmp/ipykernel_71766/580155178.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assistsMapDF['participantsAssisted'] = assistsMapDF['participantsAssisted'].apply(ast.literal_eval)
/tmp/ipykernel_71766/580155178.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assistsMapDF['towerKillsAssisted'] = assistsMapDF['towerKillsAssisted'].apply(ast.literal_eval)
/tmp/ipykernel_71766/580155178.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

,gameId,teamId,win,teamPosition,teamAverageRank,matchAverageRank,participantId,kills,deaths,assists,...,totalTeamKPM,totalTeamExpPerMin,totalTeamVPM,totalTeamAPM,totalTeamMinionsPerMin,totalAssisted,totalTeamPressureAssists,totalTeamAllAssists,totalTeamAllKills,intensity
0,EUW1_7056642171,100,0,TOP,13.0,13.8,1,7,8,2,...,1.225773,2326.139340,2.891566,1.194343,22.126768,"[2, 5, 3, 4]",14,60,47,1.276596
1,EUW1_7056642171,100,0,JUNGLE,13.0,13.8,2,8,8,11,...,1.225773,2326.139340,2.891566,1.194343,22.126768,"[1, 1, 3, 3, 3, 3, 3, 4, 4, 4, 5, 3, 7, 7, 4, ...",14,60,47,1.276596
2,EUW1_7056642171,100,0,MIDDLE,13.0,13.8,3,10,9,4,...,1.225773,2326.139340,2.891566,1.194343,22.126768,"[2, 2, 4, 4, 2, 4, 2, 2, 4, 5, 5]",14,60,47,1.276596
3,EUW1_7056642171,100,0,BOTTOM,13.0,13.8,4,10,9,9,...,1.225773,2326.139340,2.891566,1.194343,22.126768,"[2, 2, 2, 2, 3, 5, 5, 5, 5, 5, 2, 7, 3]",14,60,47,1.276596
4,EUW1_7056642171,100,0,UTILITY,13.0,13.8,5,4,7,12,...,1.225773,2326.139340,2.891566,1.194343,22.126768,"[2, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, ...",14,60,47,1.276596
5,EUW1_7056642171,200,1,TOP,14.6,13.8,6,3,10,9,...,1.288633,2463.834468,4.651650,2.168675,20.146674,"[7, 7, 7, 7, 8, 8, 9, 9, 9, 9, 7, 9]",13,94,53,1.773585
6,EUW1_7056642171,200,1,JUNGLE,14.6,13.8,7,13,7,13,...,1.288633,2463.834468,4.651650,2.168675,20.146674,"[8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 10, 10, 2, 3...",13,94,53,1.773585
7,EUW1_7056642171,200,1,MIDDLE,14.6,13.8,8,7,10,14,...,1.288633,2463.834468,4.651650,2.168675,20.146674,"[7, 7, 7, 7, 7, 7, 7, 7, 7, 9, 9, 9, 10, 10, 7...",13,94,53,1.773585
8,EUW1_7056642171,200,1,BOTTOM,14.6,13.8,9,16,8,10,...,1.288633,2463.834468,4.651650,2.168675,20.146674,"[7, 7, 7, 7, 7, 7, 8, 8, 8, 10, 6, 8, 7, 7]",13,94,53,1.773585
9,EUW1_7056642171,200,1,UTILITY,14.6,13.8,10,2,4,23,...,1.288633,2463.834468,4.651650,2.168675,20.146674,"[7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, ...",13,94,53,1.773585


In [9]:
## based on the totalAssisted column, create another column that holds a dictionary such that (key,value) such that 
# key = participantId that was assisted and 
# value = number of times that participant was assisted

def convertListToDict(list):
    myDict = {}
    for item in list:
        if item not in myDict.keys():
            myDict[item] = 1
        else:
            myDict[item] += 1
    return myDict

assistsMapDF['totalAssistsMap'] = assistsMapDF['totalAssisted'].apply(convertListToDict)
assistsMapDF


,gameId,teamId,win,teamPosition,teamAverageRank,matchAverageRank,participantId,kills,deaths,assists,...,totalTeamExpPerMin,totalTeamVPM,totalTeamAPM,totalTeamMinionsPerMin,totalAssisted,totalTeamPressureAssists,totalTeamAllAssists,totalTeamAllKills,intensity,totalAssistsMap
0,EUW1_7056642171,100,0,TOP,13.0,13.8,1,7,8,2,...,2326.13934,2.891566,1.194343,22.126768,"[2, 5, 3, 4]",14,60,47,1.276596,"{2: 1, 5: 1, 3: 1, 4: 1}"
1,EUW1_7056642171,100,0,JUNGLE,13.0,13.8,2,8,8,11,...,2326.13934,2.891566,1.194343,22.126768,"[1, 1, 3, 3, 3, 3, 3, 4, 4, 4, 5, 3, 7, 7, 4, ...",14,60,47,1.276596,"{1: 2, 3: 6, 4: 4, 5: 3, 7: 2}"
2,EUW1_7056642171,100,0,MIDDLE,13.0,13.8,3,10,9,4,...,2326.13934,2.891566,1.194343,22.126768,"[2, 2, 4, 4, 2, 4, 2, 2, 4, 5, 5]",14,60,47,1.276596,"{2: 5, 4: 4, 5: 2}"
3,EUW1_7056642171,100,0,BOTTOM,13.0,13.8,4,10,9,9,...,2326.13934,2.891566,1.194343,22.126768,"[2, 2, 2, 2, 3, 5, 5, 5, 5, 5, 2, 7, 3]",14,60,47,1.276596,"{2: 5, 3: 2, 5: 5, 7: 1}"
4,EUW1_7056642171,100,0,UTILITY,13.0,13.8,5,4,7,12,...,2326.13934,2.891566,1.194343,22.126768,"[2, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, ...",14,60,47,1.276596,"{2: 5, 3: 3, 4: 9}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34885,EUW1_7049835170,200,0,JUNGLE,15.4,14.4,6,1,10,11,...,1993.60735,5.077369,1.102515,19.119923,"[7, 7, 7, 7, 7, 7, 9, 9, 10, 10, 10, 10, 10]",9,53,28,1.892857,"{7: 6, 9: 2, 10: 5}"
34886,EUW1_7049835170,200,0,TOP,15.4,14.4,7,9,12,2,...,1993.60735,5.077369,1.102515,19.119923,"[10, 10, 6, 6, 10]",9,53,28,1.892857,"{10: 3, 6: 2}"
34887,EUW1_7049835170,200,0,UTILITY,15.4,14.4,8,0,8,12,...,1993.60735,5.077369,1.102515,19.119923,"[6, 7, 7, 7, 9, 9, 9, 9, 10, 10, 10, 10, 6, 6,...",9,53,28,1.892857,"{6: 3, 7: 4, 9: 4, 10: 7}"
34888,EUW1_7049835170,200,0,MIDDLE,15.4,14.4,9,4,3,7,...,1993.60735,5.077369,1.102515,19.119923,"[6, 7, 7, 7, 10, 10, 10, 6, 6, 6, 6, 6, 7]",9,53,28,1.892857,"{6: 6, 7: 4, 10: 3}"


In [10]:
# computing total indegree and total outdegree of individual players in the team
from collections import defaultdict

# default empty dictionary to use later for indegree assignment
indegree = defaultdict(int)

# Calculate playerOutdegree and update indegree counts; alternatively you can use it to using len of totalAssisted
assistsMapDF['playerTotalOutdegree'] = assistsMapDF['totalAssistsMap'].apply(lambda x: sum(x.values()))

for _, row in assistsMapDF.iterrows():
    for teammate, assist_count in row['totalAssistsMap'].items():
        indegree[(row['gameId'], teammate)] += assist_count

# Add playerIndegree to the DataFrame
assistsMapDF['playerTotalIndegree'] = assistsMapDF.apply(lambda row: indegree[(row['gameId'], row['participantId'])], axis=1)

In [11]:
## computing in-degree centrality of a team
# step 1; compute max of team in-degree
teamMaxIndegree = assistsMapDF.groupby(['gameId', 'teamId'])['playerTotalIndegree'].max().reset_index()
teamMaxIndegree.rename(columns={'playerTotalIndegree': 'teamMaxIndegree'}, inplace=True)
assistsMapDF = assistsMapDF.merge(teamMaxIndegree, on=['gameId', 'teamId'], how='inner')

# step 2; compute sum of all team members in-degree
teamTotalIndegree = assistsMapDF.groupby(['gameId', 'teamId'])['playerTotalIndegree'].sum().reset_index()
teamTotalIndegree.rename(columns={'playerTotalIndegree': 'teamTotalIndegree'}, inplace=True)
assistsMapDF = assistsMapDF.merge(teamTotalIndegree, on=['gameId', 'teamId'], how='inner')

# step 3; sum up the difference between max and individual in-degree and sum it up to another column
assistsMapDF['playerDiffMaxTeamIndegree'] = assistsMapDF['teamMaxIndegree'] - assistsMapDF['playerTotalIndegree']
playerDiffMaxTeamIndegreeSum = assistsMapDF.groupby(['gameId', 'teamId'])['playerDiffMaxTeamIndegree'].sum().reset_index()
playerDiffMaxTeamIndegreeSum.rename(columns={'playerDiffMaxTeamIndegree': 'playerDiffMaxTeamIndegreeSum'}, inplace=True)
assistsMapDF = assistsMapDF.merge(playerDiffMaxTeamIndegreeSum, on=['gameId', 'teamId'], how='inner')

# step 4; the summed up column to be divided by (4 * sum of all indegrees)
assistsMapDF['teamIndegreeCentrality'] = assistsMapDF['playerDiffMaxTeamIndegreeSum'] / ( 4 * assistsMapDF['teamTotalIndegree'] )

In [12]:
## computing out-degree centrality of a team and weight centralization of the team
# step 1; compute max of team's out-degree
teamMaxOutdegree = assistsMapDF.groupby(['gameId', 'teamId'])['playerTotalOutdegree'].max().reset_index()
teamMaxOutdegree.rename(columns={'playerTotalOutdegree': 'teamMaxOutdegree'}, inplace=True)
assistsMapDF = assistsMapDF.merge(teamMaxOutdegree, on=['gameId', 'teamId'], how='inner')

# step 2; compute sum of all team members out-degree
teamTotalOutdegree = assistsMapDF.groupby(['gameId', 'teamId'])['playerTotalOutdegree'].sum().reset_index()
teamTotalOutdegree.rename(columns={'playerTotalOutdegree': 'teamTotalOutdegree'}, inplace=True)
assistsMapDF = assistsMapDF.merge(teamTotalOutdegree, on=['gameId', 'teamId'], how='inner')

# step 3; sum up the difference between max and individual out-degree and sum it up to another column
assistsMapDF['playerDiffMaxTeamOutdegree'] = assistsMapDF['teamMaxOutdegree'] - assistsMapDF['playerTotalOutdegree']
playerDiffMaxTeamIndegreeSum = assistsMapDF.groupby(['gameId', 'teamId'])['playerDiffMaxTeamOutdegree'].sum().reset_index()
playerDiffMaxTeamIndegreeSum.rename(columns={'playerDiffMaxTeamOutdegree': 'playerDiffMaxTeamOutdegreeSum'}, inplace=True)
assistsMapDF = assistsMapDF.merge(playerDiffMaxTeamIndegreeSum, on=['gameId', 'teamId'], how='inner')

# step 4; the summed up column to be divided by (4 * sum of all indegrees)
assistsMapDF['teamOutdegreeCentrality'] = assistsMapDF['playerDiffMaxTeamOutdegreeSum'] / ( 4 * assistsMapDF['teamTotalIndegree'] ) # 4 = N-1; totalTeamIndegree is the same as totalTeamOutdegree; so you can use either of them

# for weight centralization
assistsMapDF['teamWeightCentralization'] = assistsMapDF['playerDiffMaxTeamOutdegreeSum'] / (19 * assistsMapDF['teamTotalIndegree'])

assistsMapDF

,gameId,teamId,win,teamPosition,teamAverageRank,matchAverageRank,participantId,kills,deaths,assists,...,teamTotalIndegree,playerDiffMaxTeamIndegree,playerDiffMaxTeamIndegreeSum,teamIndegreeCentrality,teamMaxOutdegree,teamTotalOutdegree,playerDiffMaxTeamOutdegree,playerDiffMaxTeamOutdegreeSum,teamOutdegreeCentrality,teamWeightCentralization
0,EUW1_7056642171,100,0,TOP,13.0,13.8,1,7,8,2,...,61,16,29,0.118852,17,62,13,23,0.094262,0.019845
1,EUW1_7056642171,100,0,JUNGLE,13.0,13.8,2,8,8,11,...,61,1,29,0.118852,17,62,0,23,0.094262,0.019845
2,EUW1_7056642171,100,0,MIDDLE,13.0,13.8,3,10,9,4,...,61,5,29,0.118852,17,62,6,23,0.094262,0.019845
3,EUW1_7056642171,100,0,BOTTOM,13.0,13.8,4,10,9,9,...,61,0,29,0.118852,17,62,4,23,0.094262,0.019845
4,EUW1_7056642171,100,0,UTILITY,13.0,13.8,5,4,7,12,...,61,7,29,0.118852,17,62,0,23,0.094262,0.019845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34885,EUW1_7049835170,200,0,JUNGLE,15.4,14.4,6,1,10,11,...,59,3,31,0.131356,18,59,5,31,0.131356,0.027654
34886,EUW1_7049835170,200,0,TOP,15.4,14.4,7,9,12,2,...,59,0,31,0.131356,18,59,13,31,0.131356,0.027654
34887,EUW1_7049835170,200,0,UTILITY,15.4,14.4,8,0,8,12,...,59,18,31,0.131356,18,59,0,31,0.131356,0.027654
34888,EUW1_7049835170,200,0,MIDDLE,15.4,14.4,9,4,3,7,...,59,10,31,0.131356,18,59,5,31,0.131356,0.027654


In [13]:
assistsMapDF[assistsMapDF['gameId'] == 'EUW1_6895619260']

,gameId,teamId,win,teamPosition,teamAverageRank,matchAverageRank,participantId,kills,deaths,assists,...,teamTotalIndegree,playerDiffMaxTeamIndegree,playerDiffMaxTeamIndegreeSum,teamIndegreeCentrality,teamMaxOutdegree,teamTotalOutdegree,playerDiffMaxTeamOutdegree,playerDiffMaxTeamOutdegreeSum,teamOutdegreeCentrality,teamWeightCentralization
6850,EUW1_6895619260,100,1,TOP,29.2,28.8,1,20,7,12,...,112,0,88,0.196429,29,112,12,33,0.073661,0.015508
6851,EUW1_6895619260,100,1,JUNGLE,29.2,28.8,2,13,8,10,...,112,7,88,0.196429,29,112,8,33,0.073661,0.015508
6852,EUW1_6895619260,100,1,BOTTOM,29.2,28.8,3,4,10,11,...,112,31,88,0.196429,29,112,6,33,0.073661,0.015508
6853,EUW1_6895619260,100,1,MIDDLE,29.2,28.8,4,5,10,14,...,112,38,88,0.196429,29,112,7,33,0.073661,0.015508
6854,EUW1_6895619260,100,1,UTILITY,29.2,28.8,5,8,7,22,...,112,12,88,0.196429,29,112,0,33,0.073661,0.015508
6855,EUW1_6895619260,200,0,TOP,28.4,28.8,6,2,10,14,...,96,42,139,0.361979,29,96,10,49,0.127604,0.026864
6856,EUW1_6895619260,200,0,JUNGLE,28.4,28.8,7,5,11,11,...,96,31,139,0.361979,29,96,10,49,0.127604,0.026864
6857,EUW1_6895619260,200,0,MIDDLE,28.4,28.8,8,12,9,6,...,96,27,139,0.361979,29,96,15,49,0.127604,0.026864
6858,EUW1_6895619260,200,0,BOTTOM,28.4,28.8,9,20,11,12,...,96,0,139,0.361979,29,96,14,49,0.127604,0.026864
6859,EUW1_6895619260,200,0,UTILITY,28.4,28.8,10,3,9,24,...,96,39,139,0.361979,29,96,0,49,0.127604,0.026864


In [14]:
# just create a dataframe with participant id and their positions
teamPositionsDF = assistsMapDF[['gameId', 'teamId', 'participantId', 'teamPosition']]
teamPositionsDF.drop_duplicates(inplace=True)
print(teamPositionsDF.shape)

(34890, 4)


/tmp/ipykernel_71766/130268533.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teamPositionsDF.drop_duplicates(inplace=True)


In [15]:
assistsMapDF.columns

Index(['gameId', 'teamId', 'win', 'teamPosition', 'teamAverageRank',
       'matchAverageRank', 'participantId', 'kills', 'deaths', 'assists',
       'participantsAssisted', 'towerKillsAssisted', 'monsterKillsAssisted',
       'participantsAssistedWithPressure', 'totalTeamKills',
       'totalTeamAssists', 'totalTeamTurretKills', 'totalTeamEpicMonsterKills',
       'totalTeamGPM', 'totalTeamKPM', 'totalTeamExpPerMin', 'totalTeamVPM',
       'totalTeamAPM', 'totalTeamMinionsPerMin', 'totalAssisted',
       'totalTeamPressureAssists', 'totalTeamAllAssists', 'totalTeamAllKills',
       'intensity', 'totalAssistsMap', 'playerTotalOutdegree',
       'playerTotalIndegree', 'teamMaxIndegree', 'teamTotalIndegree',
       'playerDiffMaxTeamIndegree', 'playerDiffMaxTeamIndegreeSum',
       'teamIndegreeCentrality', 'teamMaxOutdegree', 'teamTotalOutdegree',
       'playerDiffMaxTeamOutdegree', 'playerDiffMaxTeamOutdegreeSum',
       'teamOutdegreeCentrality', 'teamWeightCentralization'],
      dt

In [16]:
# expanding the totalAssistsMap columns to reflect the weights between nodes of players
edgesDF_rows = []

# Iterate through each row in the DataFrame
for _, row in assistsMapDF.iterrows():
    gameId = row['gameId']
    teamId = row['teamId']
    teamAverageRank = row['teamAverageRank']
    matchAverageRank = row['matchAverageRank']
    participant = row['participantId']
    totalAssistsMap = row['totalAssistsMap']
    teamIndegreeCentrality = row['teamIndegreeCentrality']
    teamOutdegreeCentrality = row['teamOutdegreeCentrality']
    teamTotalAllAssists = row['totalTeamAllAssists']
    win = row['win']
    teamGPM = row['totalTeamGPM']
    teamAverageRank = row['teamAverageRank']
    matchAverageRank = row['matchAverageRank']
    
    # Iterate through each key-value pair in the totalAssistsMap
    for to_participant, weight in totalAssistsMap.items():
        edgesDF_rows.append({
            'gameId': gameId,
            'teamId': teamId,
            'win': win,
            'fromParticipantId': participant,
            'toParticipantId': to_participant,
            'weight': weight,
            'teamIndegreeCentrality': teamIndegreeCentrality,
            'teamOutdegreeCentrality': teamOutdegreeCentrality,
            'teamTotalAllAssists': teamTotalAllAssists,
            'teamGPM': teamGPM,
            'teamAverageRank': teamAverageRank,
            'matchAverageRank': matchAverageRank
        })

# Create a new DataFrame from the expanded rows
edgesDF = pd.DataFrame(edgesDF_rows)

edgesDF['uniqueTeamId'] = edgesDF['gameId'].astype(str) + '_' + edgesDF['teamId'].astype(str)
edgesDF['normalizedWeight'] = edgesDF['weight'] / edgesDF['teamTotalAllAssists']

edgesDF.head(20)

,gameId,teamId,win,fromParticipantId,toParticipantId,weight,teamIndegreeCentrality,teamOutdegreeCentrality,teamTotalAllAssists,teamGPM,teamAverageRank,matchAverageRank,uniqueTeamId,normalizedWeight
0,EUW1_7056642171,100,0,1,2,1,0.118852,0.094262,60,2024.0,13.0,13.8,EUW1_7056642171_100,0.016667
1,EUW1_7056642171,100,0,1,5,1,0.118852,0.094262,60,2024.0,13.0,13.8,EUW1_7056642171_100,0.016667
2,EUW1_7056642171,100,0,1,3,1,0.118852,0.094262,60,2024.0,13.0,13.8,EUW1_7056642171_100,0.016667
3,EUW1_7056642171,100,0,1,4,1,0.118852,0.094262,60,2024.0,13.0,13.8,EUW1_7056642171_100,0.016667
4,EUW1_7056642171,100,0,2,1,2,0.118852,0.094262,60,2024.0,13.0,13.8,EUW1_7056642171_100,0.033333
5,EUW1_7056642171,100,0,2,3,6,0.118852,0.094262,60,2024.0,13.0,13.8,EUW1_7056642171_100,0.100000
6,EUW1_7056642171,100,0,2,4,4,0.118852,0.094262,60,2024.0,13.0,13.8,EUW1_7056642171_100,0.066667
7,EUW1_7056642171,100,0,2,5,3,0.118852,0.094262,60,2024.0,13.0,13.8,EUW1_7056642171_100,0.050000
8,EUW1_7056642171,100,0,2,7,2,0.118852,0.094262,60,2024.0,13.0,13.8,EUW1_7056642171_100,0.033333
9,EUW1_7056642171,100,0,3,2,5,0.118852,0.094262,60,2024.0,13.0,13.8,EUW1_7056642171_100,0.083333


In [17]:
# merge team position of respective from participant id to participant id
edgesDF = edgesDF.merge(teamPositionsDF, left_on=['gameId', 'teamId', 'fromParticipantId'], right_on=['gameId', 'teamId', 'participantId'], how='inner')
edgesDF.rename(columns={'teamPosition': 'fromPlayerPosition'}, inplace=True)

edgesDF = edgesDF.merge(teamPositionsDF, left_on=['gameId', 'teamId', 'toParticipantId'], right_on=['gameId', 'teamId', 'participantId'], how='inner')
edgesDF.rename(columns={'teamPosition': 'toPlayerPosition'}, inplace=True)

In [18]:
# computing resistance for the team by drawing a network for each team in each match and calculate resistance
def CalcResistance(eigenvalues,N=5):
    eigval = eigenvalues.tolist()
    roundedEigval = [round(val, 2) for val in eigval]
    roundedEigval = [i for i in roundedEigval if i != 0]

    summ = 0
    for e in roundedEigval:
        summ = summ + (1/e)
    R = 5 * summ
    return R

# for calculating graph resistance of teams using the edgesDF
uniqueTeamIdList = edgesDF['uniqueTeamId'].tolist()
uniqueTeamIdList = list(set(uniqueTeamIdList))
resistanceDict = {}

for idx, uid in enumerate(uniqueTeamIdList):
    uniqueTeamDF = edgesDF[edgesDF['uniqueTeamId'] == uid]
    
    # create a network graph object using network library; compute a
    G = nx.from_pandas_edgelist(uniqueTeamDF,'fromParticipantId','toParticipantId', edge_attr='weight', create_using=nx.MultiGraph(directed=True))
    A = nx.adjacency_matrix(G) # adjacency matrix
    D = np.diag(np.sum(np.array(A.todense()), axis=1)) # degree matrix
    L = D - A # laplacian matrix
    e, v = np.linalg.eig(L) # eigen values

    resistanceValue = CalcResistance(e)
    resistanceDict[uid] = round(resistanceValue,4)

teamResistance = pd.DataFrame(resistanceDict.items(), columns=['uniqueTeamId', 'resistance'])

# merge teamResistance DF into edgesDF using uniqueTeamId key
edgesDF = edgesDF.merge(teamResistance, on='uniqueTeamId', how='inner')
print(edgesDF.shape)

# merge teamResistance to assistsMapDF
assistsMapDF['uniqueTeamId'] = assistsMapDF['gameId'].astype(str) + '_' + assistsMapDF['teamId'].astype(str)
assistsMapDF = assistsMapDF.merge(teamResistance, on='uniqueTeamId', how='inner')
assistsMapDF.head(10)


(117311, 19)


,gameId,teamId,win,teamPosition,teamAverageRank,matchAverageRank,participantId,kills,deaths,assists,...,playerDiffMaxTeamIndegreeSum,teamIndegreeCentrality,teamMaxOutdegree,teamTotalOutdegree,playerDiffMaxTeamOutdegree,playerDiffMaxTeamOutdegreeSum,teamOutdegreeCentrality,teamWeightCentralization,uniqueTeamId,resistance
0,EUW1_7056642171,100,0,TOP,13.0,13.8,1,7,8,2,...,29,0.118852,17,62,13,23,0.094262,0.019845,EUW1_7056642171_100,1.1017
1,EUW1_7056642171,100,0,JUNGLE,13.0,13.8,2,8,8,11,...,29,0.118852,17,62,0,23,0.094262,0.019845,EUW1_7056642171_100,1.1017
2,EUW1_7056642171,100,0,MIDDLE,13.0,13.8,3,10,9,4,...,29,0.118852,17,62,6,23,0.094262,0.019845,EUW1_7056642171_100,1.1017
3,EUW1_7056642171,100,0,BOTTOM,13.0,13.8,4,10,9,9,...,29,0.118852,17,62,4,23,0.094262,0.019845,EUW1_7056642171_100,1.1017
4,EUW1_7056642171,100,0,UTILITY,13.0,13.8,5,4,7,12,...,29,0.118852,17,62,0,23,0.094262,0.019845,EUW1_7056642171_100,1.1017
5,EUW1_7056642171,200,1,TOP,14.6,13.8,6,3,10,9,...,93,0.239691,30,96,18,54,0.139175,0.029300,EUW1_7056642171_200,0.5957
6,EUW1_7056642171,200,1,JUNGLE,14.6,13.8,7,13,7,13,...,93,0.239691,30,96,11,54,0.139175,0.029300,EUW1_7056642171_200,0.5957
7,EUW1_7056642171,200,1,MIDDLE,14.6,13.8,8,7,10,14,...,93,0.239691,30,96,9,54,0.139175,0.029300,EUW1_7056642171_200,0.5957
8,EUW1_7056642171,200,1,BOTTOM,14.6,13.8,9,16,8,10,...,93,0.239691,30,96,16,54,0.139175,0.029300,EUW1_7056642171_200,0.5957
9,EUW1_7056642171,200,1,UTILITY,14.6,13.8,10,2,4,23,...,93,0.239691,30,96,0,54,0.139175,0.029300,EUW1_7056642171_200,0.5957


In [19]:
def convertNumericalRank(nRank):
    if nRank > 30:
        return 'CHALLENGER'
    elif nRank <=30 and nRank > 29:
        return 'GRANDMASTER'
    elif nRank <= 29 and nRank > 28:
        return 'MASTER'
    elif nRank <= 28 and nRank > 24:
        return 'DIAMOND'
    elif nRank <= 24 and nRank > 20:
        return 'EMERALD'
    elif nRank <= 20 and nRank > 16:
        return 'PLATINUM'
    elif nRank <= 16 and nRank > 12:
        return 'GOLD'
    elif nRank <= 12 and nRank > 8:
        return 'SILVER'
    elif nRank <= 8 and nRank > 4:
        return 'BRONZE'
    else:
        return 'IRON'
    
edgesDF['gameRank'] = edgesDF['matchAverageRank'].apply(convertNumericalRank)
edgesDF['teamRank'] = edgesDF['teamAverageRank'].apply(convertNumericalRank)

print(edgesDF.columns)

Index(['gameId', 'teamId', 'win', 'fromParticipantId', 'toParticipantId',
       'weight', 'teamIndegreeCentrality', 'teamOutdegreeCentrality',
       'teamTotalAllAssists', 'teamGPM', 'teamAverageRank', 'matchAverageRank',
       'uniqueTeamId', 'normalizedWeight', 'participantId_x',
       'fromPlayerPosition', 'participantId_y', 'toPlayerPosition',
       'resistance', 'gameRank', 'teamRank'],
      dtype='object')


In [20]:
gameIds = edgesDF['gameId'].tolist()
gameIds = list(set(gameIds))
gameIds = sorted(gameIds)

# commented the below which generate match graphs and stores in locally; dont want it to run every time I do some other analysis
'''
# use incremental ranges of gameIds to save figure locally
for idx, gameId in enumerate(gameIds[3200:3490]):
    # team level data of two teams in the same match-up
    blueTeamDF = edgesDF[(edgesDF['gameId'] == gameId) & (edgesDF['teamId'] == 100)]
    blueTeamDF = blueTeamDF[(blueTeamDF['toParticipantId'] >= 1) & (blueTeamDF['toParticipantId'] <= 5)] # additional filter to remove any steals or lease

    redTeamDF = edgesDF[(edgesDF['gameId'] == gameId) & (edgesDF['teamId'] == 200)]
    redTeamDF = redTeamDF[(redTeamDF['toParticipantId'] >= 6) & (redTeamDF['toParticipantId'] <= 10)] # additional filter to remove any steal or lease

    blueTeamEdges = [ min(100 * i, 5) for i in blueTeamDF['normalizedWeight'] ]
    redTeamEdges = [ min(100 * i, 5) for i in redTeamDF['normalizedWeight'] ]

    matchRank = blueTeamDF['gameRank'].iloc[0]
    titleText = f'Match ID: {gameId}    Match Tier: {matchRank}'

    blueTeamAvgRank = blueTeamDF['teamRank'].iloc[0]
    blueTeamWins = blueTeamDF['win'].iloc[0]
    blueTeamGPM = blueTeamDF['teamGPM'].iloc[0]
    blueTeamInD = blueTeamDF['teamIndegreeCentrality'].iloc[0]
    blueTeamOutD = blueTeamDF['teamOutdegreeCentrality'].iloc[0]
    blueTeamRes = blueTeamDF['resistance'].iloc[0]
    
    blueTeamText = f'WIN: {blueTeamWins} | Team Rank: {blueTeamAvgRank} | Gold Per Min.: {blueTeamGPM} \n In-Degree: {blueTeamInD:.3f} | Out-Degree: {blueTeamOutD:.3f} | Resistance: {blueTeamRes:.3f}'
    
    redTeamAvgRank = redTeamDF['teamRank'].iloc[0]
    redTeamWins = redTeamDF['win'].iloc[0]
    redTeamGPM = redTeamDF['teamGPM'].iloc[0]
    redTeamGPM = redTeamDF['teamGPM'].iloc[0]
    redTeamInD = redTeamDF['teamIndegreeCentrality'].iloc[0]
    redTeamOutD = redTeamDF['teamOutdegreeCentrality'].iloc[0]
    redTeamRes = redTeamDF['resistance'].iloc[0]

    redTeamText = f'WIN: {redTeamWins} | Team Rank: {redTeamAvgRank} | Gold Per Min.: {redTeamGPM} \n In-Degree: {redTeamInD:.3f} | Out-Degree: {redTeamOutD:.3f} | Resistance: {redTeamRes:.3f}'

    # create network graph objects
    BTG = nx.from_pandas_edgelist(blueTeamDF, source='fromPlayerPosition', target='toPlayerPosition', edge_attr='normalizedWeight', create_using=nx.MultiDiGraph(directed=True))
    RTG = nx.from_pandas_edgelist(redTeamDF, source='fromPlayerPosition', target='toPlayerPosition', edge_attr='normalizedWeight', create_using=nx.MultiDiGraph(directed=True))

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    posB = nx.circular_layout(BTG)
    posR = nx.circular_layout(RTG)
    
    nx.draw(BTG, pos=posB, ax=axes[0], with_labels=True, node_size=2100, node_color="lightblue", font_size=8, connectionstyle='arc3,rad=0.1', font_weight="bold", arrowsize=8, width=blueTeamEdges, edge_color='gray')
    axes[0].set_title(blueTeamText)

    nx.draw(RTG, pos=posR, ax=axes[1], with_labels=True, node_size=2100, node_color="red", font_size=8, connectionstyle='arc3,rad=0.1', font_weight="bold", arrowsize=8, width=redTeamEdges, edge_color='gray')
    axes[1].set_title(redTeamText)

    # Match title
    plt.suptitle(titleText, fontsize=16)
    # plt.tight_layout()
    # plt.show()

    # save the figure
    output_path = f'../graphs/{matchRank}/{gameId}.png'
    plt.savefig(output_path, format='png', dpi=300)
    print(f"Image {output_path} saved successfully")
    plt.clf()
'''

'\n# use incremental ranges of gameIds to save figure locally\nfor idx, gameId in enumerate(gameIds[3200:3490]):\n    # team level data of two teams in the same match-up\n    blueTeamDF = edgesDF[(edgesDF[\'gameId\'] == gameId) & (edgesDF[\'teamId\'] == 100)]\n    blueTeamDF = blueTeamDF[(blueTeamDF[\'toParticipantId\'] >= 1) & (blueTeamDF[\'toParticipantId\'] <= 5)] # additional filter to remove any steals or lease\n\n    redTeamDF = edgesDF[(edgesDF[\'gameId\'] == gameId) & (edgesDF[\'teamId\'] == 200)]\n    redTeamDF = redTeamDF[(redTeamDF[\'toParticipantId\'] >= 6) & (redTeamDF[\'toParticipantId\'] <= 10)] # additional filter to remove any steal or lease\n\n    blueTeamEdges = [ min(100 * i, 5) for i in blueTeamDF[\'normalizedWeight\'] ]\n    redTeamEdges = [ min(100 * i, 5) for i in redTeamDF[\'normalizedWeight\'] ]\n\n    matchRank = blueTeamDF[\'gameRank\'].iloc[0]\n    titleText = f\'Match ID: {gameId}    Match Tier: {matchRank}\'\n\n    blueTeamAvgRank = blueTeamDF[\'teamRank\

In [21]:
print(valid_player_match_details_df.shape)
print(valid_player_match_details_df.columns)

(34890, 56)
Index(['gameId', 'gameDuration', 'puuid', 'summonerId', 'teamId', 'win',
       'teamPosition', 'champExperience', 'kills', 'assists', 'turretKills',
       'epicMonsterKills', 'visionScore', 'visionWardsBoughtInGame',
       'magicDamageDealtToChampions', 'deaths', 'totalMinionsKilled',
       'timeCCingOthers', 'totalDamageTaken', 'totalHealsOnTeammates',
       'totalTimeSpentDead', 'goldEarned', 'objectivesStolen',
       'objectivesStolenAssists', 'participantId', 'participantsAssisted',
       'towerKillsAssisted', 'monsterKillsAssisted',
       'participantsAssistedWithPressure', 'queueType', 'tier', 'rank',
       'leaguePoints', 'matchPlayerCounts', 'tierRank', 'playerNumericalRank',
       'matchAverageRank', 'teamAverageRank', 'totalTeamGold', 'totalTeamGPM',
       'totalTeamKills', 'totalTeamKPM', 'totalTeamDeaths', 'totalTeamAssists',
       'totalTeamAPM', 'totalTeamEpicMonsterKills', 'totalTeamTurretKills',
       'totalTeamVisionScore', 'totalTeamVPM', 'ave

In [22]:
df = assistsMapDF[['gameId', 'teamId', 'participantId', 'intensity', 'totalTeamAllKills', 'totalTeamAllAssists', 'playerTotalIndegree', 'playerTotalOutdegree', 'teamIndegreeCentrality', 'teamOutdegreeCentrality', 'teamWeightCentralization', 'resistance']]
df.drop_duplicates(inplace=True)
print(df.shape)

valid_player_match_details_df = valid_player_match_details_df.merge(df, on=['gameId', 'teamId', 'participantId'], how='inner')
print(valid_player_match_details_df.shape)


(34890, 12)
(34890, 65)


/tmp/ipykernel_71766/2499663115.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


### Export team level statistics to a CSV

In [23]:
from sklearn.preprocessing import StandardScaler

# team averaged dataset
teamDF = valid_player_match_details_df[[
    'gameId', 'teamId', 'gameDuration', 'win', 'teamAverageRank', 'matchAverageRank', 
    'totalTeamKills', 'totalTeamEpicMonsterKills', 'totalTeamTurretKills', 'totalTeamAllAssists', 'totalTeamGold', 'totalTeamVisionScore', 'totalTeamChampExperience', 
    'totalTeamMinionsKilled', 'resistance', 'teamIndegreeCentrality', 'teamOutdegreeCentrality', 'teamWeightCentralization'
    ]]

teamDF.drop_duplicates(inplace=True) # redundant rows for 5 teammates

teamDF['goldPerMin'] = teamDF['totalTeamGold'] / teamDF['gameDuration']
teamDF['totalTeamMininionsPerMin'] = teamDF['totalTeamMinionsKilled'] / teamDF['gameDuration']
teamDF['totalTeamVisionPerMin'] = teamDF['totalTeamVisionScore'] / teamDF['gameDuration']
teamDF['totalTeamChampExpPerMin'] = teamDF['totalTeamChampExperience'] / teamDF['gameDuration']
teamDF['totalTeamAllAssistsPerMin'] = teamDF['totalTeamAllAssists'] / teamDF['gameDuration']

teamDFAllMetrics = teamDF.copy()
teamDFAllMetrics.to_csv('./teamAllMetrics.csv', index=False)

/tmp/ipykernel_71766/214268116.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teamDF.drop_duplicates(inplace=True) # redundant rows for 5 teammates
/tmp/ipykernel_71766/214268116.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teamDF['goldPerMin'] = teamDF['totalTeamGold'] / teamDF['gameDuration']
/tmp/ipykernel_71766/214268116.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [24]:
for col in assistsMapDF.columns:
    print(col)

gameId
teamId
win
teamPosition
teamAverageRank
matchAverageRank
participantId
kills
deaths
assists
participantsAssisted
towerKillsAssisted
monsterKillsAssisted
participantsAssistedWithPressure
totalTeamKills
totalTeamAssists
totalTeamTurretKills
totalTeamEpicMonsterKills
totalTeamGPM
totalTeamKPM
totalTeamExpPerMin
totalTeamVPM
totalTeamAPM
totalTeamMinionsPerMin
totalAssisted
totalTeamPressureAssists
totalTeamAllAssists
totalTeamAllKills
intensity
totalAssistsMap
playerTotalOutdegree
playerTotalIndegree
teamMaxIndegree
teamTotalIndegree
playerDiffMaxTeamIndegree
playerDiffMaxTeamIndegreeSum
teamIndegreeCentrality
teamMaxOutdegree
teamTotalOutdegree
playerDiffMaxTeamOutdegree
playerDiffMaxTeamOutdegreeSum
teamOutdegreeCentrality
teamWeightCentralization
uniqueTeamId
resistance


## Individual Statistics

In [25]:
playerDF = valid_player_match_details_df[['gameId', 'teamId', 'participantId', 'teamPosition', 'win', 'gameDuration', 'playerNumericalRank', 'goldEarned',
                                           'champExperience', 'kills', 'deaths', 'assists', 'turretKills', 'epicMonsterKills', 
                                           'visionScore', 'totalMinionsKilled', 'playerTotalIndegree', 'playerTotalOutdegree']]

playerDF['goldPerMin'] = playerDF['goldEarned'] / playerDF['gameDuration']
playerDF['totalMinionsPerMin'] = playerDF['totalMinionsKilled'] / playerDF['gameDuration']
playerDF['visionScorePerMin'] = playerDF['visionScore'] / playerDF['gameDuration']
playerDF['champExpPerMin'] = playerDF['champExperience'] / playerDF['gameDuration']

playerDF.to_csv('./playerAllMetrics.csv', index=False)

playerDF.head(10)


/tmp/ipykernel_71766/4089252784.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playerDF['goldPerMin'] = playerDF['goldEarned'] / playerDF['gameDuration']
/tmp/ipykernel_71766/4089252784.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playerDF['totalMinionsPerMin'] = playerDF['totalMinionsKilled'] / playerDF['gameDuration']
/tmp/ipykernel_71766/4089252784.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

,gameId,teamId,participantId,teamPosition,win,gameDuration,playerNumericalRank,goldEarned,champExperience,kills,...,turretKills,epicMonsterKills,visionScore,totalMinionsKilled,playerTotalIndegree,playerTotalOutdegree,goldPerMin,totalMinionsPerMin,visionScorePerMin,champExpPerMin
0,EUW1_7056642171,100,1,TOP,0,31.816667,13,13186,17614,7,...,2,0,18,238,2,4,414.436878,7.480356,0.565741,553.609219
1,EUW1_7056642171,100,2,JUNGLE,0,31.816667,13,14054,16470,8,...,1,1,27,53,17,17,441.718177,1.665794,0.848612,517.653222
2,EUW1_7056642171,100,3,MIDDLE,0,31.816667,14,12779,13825,10,...,1,0,15,163,13,11,401.644840,5.123101,0.471451,434.520691
3,EUW1_7056642171,100,4,BOTTOM,0,31.816667,12,14540,14832,10,...,3,0,13,208,18,13,456.993190,6.537454,0.408591,466.170770
4,EUW1_7056642171,100,5,UTILITY,0,31.816667,13,9820,11269,4,...,0,0,19,42,11,17,308.643269,1.320063,0.597171,354.185437
5,EUW1_7056642171,200,6,TOP,1,31.816667,14,14549,17893,3,...,4,0,22,220,1,12,457.276061,6.914615,0.691461,562.378208
6,EUW1_7056642171,200,7,JUNGLE,1,31.816667,22,15430,15110,13,...,0,3,24,53,38,19,484.965951,1.665794,0.754322,474.908329
7,EUW1_7056642171,200,8,MIDDLE,1,31.816667,13,11816,14917,7,...,0,1,20,129,17,21,371.377685,4.054479,0.628601,468.842326
8,EUW1_7056642171,200,9,BOTTOM,1,31.816667,13,16098,16250,16,...,4,0,12,212,36,14,505.961236,6.663174,0.377161,510.738607
9,EUW1_7056642171,200,10,UTILITY,1,31.816667,11,10013,14221,2,...,0,0,70,27,5,30,314.709272,0.848612,2.200105,446.966998
